# **IP1_Run2**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.6 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Get

In [2]:

import openai
import os
from IPython.display import Markdown


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']



### **Generate Mathematical Model**

In [4]:
problem = """We are delighted to welcome you, our newest intern on the Analytics team of Massachusetts General Hospital! You have been placed in a challenging role where you will be tasked with solving a real-world problem in the field of medical physics. We are building a pilot program in Boston, and if successful, your work could be applied widely in hospitals with limited capacity in many countries.

You are responsible for determining the best treatment plan for 17 patients who require radiotherapy. Your goal is to optimize the use of two possible treatments: photon therapy and proton therapy. While proton therapy is known to target tumors more precisely, it is also more expensive and has limited capacity in many countries. Therefore, you will need to balance the benefits of proton therapy with its limitations and cost to create an effective treatment plan for each patient.

To determine the best course of action for each patient, you will use a scoring system called the Biological Equivalent Dose (BED). This system allows you to calculate the effectiveness of each patient’s treatment plan by considering the number of proton fractions that can be used while still achieving the highest possible BED.

We have n=17 patients who need radiotherapy. Each patient i needs 15 fractions, which can be photon fractions, proton fractions, or a mix of photon and proton fractions (e.g. 4 proton fractions and 11 photon fractions). We want to use the limited proton therapy capacity as best as possible. We can calculate the BED score for each patient when p proton fractions and 15-p photon fractions are used, as BEDi(p,15-p), i.e., the BED when p proton and 15-p photon fractions are delivered for patient i. The higher the score, the better.

The data file "ProblemData.csv" contains a 2D matrix of BED scores. It does not have an index. It was made in Excel and saved as csv. The columns are the number of proton fractions and each row represents a patient. In particular, the number at the (i,j) position is the score for patient i receiving j proton fractions.

Suppose that the total maximal capacity C is 100 proton fractions. To maximize the total BED scores for all the patients, which patients should get proton fractions, and how many should they get? Formulate an integer linear optimization model to solve this problem. Assume you know the value BEDi(j,15-j) for each patient i.  """


In [6]:

client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the variables for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 2
)


In [7]:
print(response1.choices[0].message.content)

Let:
- i = index for patients, where i ∈ {1,2,...,17}
- j = index for the number of proton fractions, where j ∈ {0,1,...,15}
- BEDi,j = BED score of patient i when j proton fractions and (15-j) photon fractions are used, read from "ProblemData.csv" 
- C = maximum capacity of proton fractions = 100
- Xij = the number of proton fractions delivered to patient i ∈ {1,2,...,17}, where Xij ∈ {0,1,...,15}

We assume that BEDi,j are known for each patient i, thus the values of Xij are the decision variables.


In [8]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the objective function for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [9]:
Markdown(response2.choices[0].message.content)

Maximize ∑ (from i=1 to 17) ∑ (from j=0 to 15) BEDi,j * Xij

Subject to

∑ (from j=0 to 15) Xij ≤ C for all i ∈ {1,2,...,17}

In [10]:
response3 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the constraints for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content +  response2.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [11]:
Markdown(response3.choices[0].message.content)

The constraints for this problem will be:

1) Each patient can receive a maximum of 15 fractions. This can be represented as:
∑ (from j=0 to 15) Xij = 15 for all i ∈ {1,2,...,17}

2) The total number of proton fractions delivered across all patients cannot exceed the total capacity C. This can be represented as:
∑ (from i=1 to 17) ∑ (from j=0 to 15) Xij ≤ C

3) The number of proton fractions delivered to each patient should be non-negative integers. This can be represented as:
Xij ∈ {0,1,...,15} for all i ∈ {1,2,...,17} and j ∈ {0,1,...,15}

### **Generate Pyomo Code**

In [12]:
response4 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content + response2.choices[0].message.content + response3.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [13]:
Markdown(response4.choices[0].message.content)

Here is the Pyomo code for this optimization problem:

```python
from pyomo.environ import *

# Create a simple model
model = ConcreteModel()

# For this example I'm using sample data
patient_num = 3
max_fraction = 5
proton_capacity = 10

# Caculate the range for this problem
I = range(1, patient_num + 1)
J = range(0, max_fraction + 1)

# Create Sample BED data
BED_data = {
    (1, 0): 10.0, (1, 1): 20.0, (1, 2): 30.0, (1, 3): 40.0, (1, 4): 50.0, (1, 5): 60.0,
    (2, 0): 15.0, (2, 1): 25.0, (2, 2): 35.0, (2, 3): 45.0, (2, 4): 55.0, (2, 5): 65.0,
    (3, 0): 20.0, (3, 1): 30.0, (3, 2): 40.0, (3, 3): 50.0, (3, 4): 60.0, (3, 5): 70.0 
}

# Sets
model.patients = Set(initialize=I)
model.fractions = Set(initialize=J)

# Parameters
model.BED = Param(model.patients, model.fractions, initialize=BED_data)
model.protonCap = Param(initialize=proton_capacity)

# Variables
model.X = Var(model.patients, model.fractions, domain=NonNegativeIntegers)

# Objective
model.obj = Objective(expr = sum(model.BED[i,j]*model.X[i,j] for i in model.patients for j in model.fractions), sense=maximize)

# Constraints
model.capacityConstr = Constraint(expr = sum(model.X[i,j] for i in model.patients for j in model.fractions)<=model.protonCap)
model.fractionConstr = ConstraintList()
for i in model.patients:
    model.fractionConstr.add(sum(model.X[i,j] for j in model.fractions)==max_fraction)

# Initialize optimizer
opt = SolverFactory('glpk')

# Solve the model
opt.solve(model)

# Print the results
for i in model.patients:
    for j in model.fractions: 
        print('Patient:', i, 'Fraction:', j, 'X:', model.X[i,j].value)
```

In this script, BED_data is your sample data. You can read from your DataFrame and assign to BED_data dict in the actual problem.

In this example, we restrict the problem to 3 patients (I = {1,2,3}) and 5 fractions (J = {0,1,...,5}) for simplicity. You may substitute I and J with the actual patients and fractions data, and update the constraints and objective correspondingly.

### **Run the code Generated by GPT4**

In [14]:
from pyomo.environ import *

# Create a simple model
model = ConcreteModel()

# For this example I'm using sample data
patient_num = 3
max_fraction = 5
proton_capacity = 10

# Caculate the range for this problem
I = range(1, patient_num + 1)
J = range(0, max_fraction + 1)

# Create Sample BED data
BED_data = {
    (1, 0): 10.0, (1, 1): 20.0, (1, 2): 30.0, (1, 3): 40.0, (1, 4): 50.0, (1, 5): 60.0,
    (2, 0): 15.0, (2, 1): 25.0, (2, 2): 35.0, (2, 3): 45.0, (2, 4): 55.0, (2, 5): 65.0,
    (3, 0): 20.0, (3, 1): 30.0, (3, 2): 40.0, (3, 3): 50.0, (3, 4): 60.0, (3, 5): 70.0
}

# Sets
model.patients = Set(initialize=I)
model.fractions = Set(initialize=J)

# Parameters
model.BED = Param(model.patients, model.fractions, initialize=BED_data)
model.protonCap = Param(initialize=proton_capacity)

# Variables
model.X = Var(model.patients, model.fractions, domain=NonNegativeIntegers)

# Objective
model.obj = Objective(expr = sum(model.BED[i,j]*model.X[i,j] for i in model.patients for j in model.fractions), sense=maximize)

# Constraints
model.capacityConstr = Constraint(expr = sum(model.X[i,j] for i in model.patients for j in model.fractions)<=model.protonCap)
model.fractionConstr = ConstraintList()
for i in model.patients:
    model.fractionConstr.add(sum(model.X[i,j] for j in model.fractions)==max_fraction)

# Initialize optimizer
opt = SolverFactory('glpk')

# Solve the model
opt.solve(model)

# Print the results
for i in model.patients:
    for j in model.fractions:
        print('Patient:', i, 'Fraction:', j, 'X:', model.X[i,j].value)

Patient: 1 Fraction: 0 X: None
Patient: 1 Fraction: 1 X: None
Patient: 1 Fraction: 2 X: None
Patient: 1 Fraction: 3 X: None
Patient: 1 Fraction: 4 X: None
Patient: 1 Fraction: 5 X: None
Patient: 2 Fraction: 0 X: None
Patient: 2 Fraction: 1 X: None
Patient: 2 Fraction: 2 X: None
Patient: 2 Fraction: 3 X: None
Patient: 2 Fraction: 4 X: None
Patient: 2 Fraction: 5 X: None
Patient: 3 Fraction: 0 X: None
Patient: 3 Fraction: 1 X: None
Patient: 3 Fraction: 2 X: None
Patient: 3 Fraction: 3 X: None
Patient: 3 Fraction: 4 X: None
Patient: 3 Fraction: 5 X: None


In [19]:
from pyomo.environ import *
import pandas as pd
# Create a simple model
model = ConcreteModel()

#LOADING THE DATA DONE BY HUMAN
patient_num = 17
max_fraction = 15
proton_capacity = 100

# Caculate the range for this problem
I = range(1, patient_num + 1)
J = range(0, max_fraction + 1)

# Create Sample BED data
df = pd.read_csv('ProblemData.csv', header=None)
BED_data= {}
for i, row in df.iterrows():
    for j, dose in enumerate(row):
        BED_data[(i+1, j)] = dose
#End

# Sets
model.patients = Set(initialize=I)
model.fractions = Set(initialize=J)

# Parameters
model.BED = Param(model.patients, model.fractions, initialize=BED_data)
model.protonCap = Param(initialize=proton_capacity)

# Variables
model.X = Var(model.patients, model.fractions, domain=NonNegativeIntegers)

# Objective
model.obj = Objective(expr = sum(model.BED[i,j]*model.X[i,j] for i in model.patients for j in model.fractions), sense=maximize)

# Constraints
model.capacityConstr = Constraint(expr = sum(model.X[i,j] for i in model.patients for j in model.fractions)<=model.protonCap)
model.fractionConstr = ConstraintList()
for i in model.patients:
    model.fractionConstr.add(sum(model.X[i,j] for j in model.fractions)==max_fraction)

# Initialize optimizer
opt = SolverFactory('glpk')

# Solve the model
opt.solve(model)

# Print the results


{'Problem': [{'Name': 'unknown', 'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 18, 'Number of variables': 272, 'Number of nonzeros': 544, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'infeasible', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.006822824478149414}]}

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**